In [69]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager

In [70]:
driver = webdriver.Chrome(ChromeDriverManager().install())

c:\Users\Nicko\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [71]:
def brand_valid(title, brand):
    
    if brand.lower() in title.lower():
        return True
    elif "VW".lower() in title.lower() and brand.lower() == "volkswagen":
        return True
    elif "Benz".lower() in title.lower() and brand.lower() == "mercedes":
        return True

    return False

In [72]:
brands = ["Audi", "BMW", "Chery", "Chevrolet", "Chrysler", "Daihatsu", "Datsun", "Ford", "Foton", "Hino", "Honda", "Hummer", "Hyundai", "Isuzu", "Jaguar", "Jeep", "Kia", "Land Rover", "Lexus", "Mazda", "Mercedes", "Mitsubishi", "Nissan", "Opel", "Peugeot", "Proton", "Renault", "SSangyong", "Subaru", "Suzuki", "Toyota", "UD Trucks", "Volkswagen", "Volvo", "Wuling", "Yamaha"]
types = ["Mobil", "Otomotif", "Bisnis"]
titles_list = []
brands_list = []
count = 0
url = "https://www.viva.co.id/search?q="
month = "Mei"
scraping_month = "Juni"
year = "2022"
SCROLL_RANGE = 3500

for brand in brands:
    driver.get(url + brand)
    content = driver.page_source
    soup = BeautifulSoup(content)
    brand_count = 0
    SCROLL_HEIGHT = 2100

    last_index = soup.find('div', attrs={'class':'article-list-container'}).findAll('div', attrs={'class':'article-list-row'})[-1].find('div', attrs={'class':'article-list-date content_center'}).text.strip()
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(5)
    driver.execute_script("window.scrollTo(0, " + str(SCROLL_HEIGHT) + ");")

    # scroll terus sampai datanya abis
    while year in last_index and (scraping_month in last_index or month in last_index):
        time.sleep(1)
        try:
            driver.find_element(By.ID, "load-more-btn").click()
        except:
            print("ERORRRRRRRR..............!!!")
            break
        content = driver.page_source
        soup = BeautifulSoup(content)
        last_index = soup.find('div', attrs={'class':'article-list-container'}).findAll('div', attrs={'class':'article-list-row'})[-1].find('div', attrs={'class':'article-list-date content_center'}).text.strip()
        
        SCROLL_HEIGHT += SCROLL_RANGE
        time.sleep(2)
        driver.execute_script("window.scrollTo(0, " + str(SCROLL_HEIGHT) + ");")
    
    for news in soup.find('div', attrs={'class':'article-list-container'}).findAll('div', attrs={'class':'article-list-row'}):
        title = news.find('a', attrs={'class':'article-list-title'}).text.strip()
        date = news.find('div', attrs={'class':'article-list-date content_center'}).text.strip()
        type = news.find('a', attrs={'class':'article-list-cate content_center'}).text.strip()

        if brand_valid(title, brand) and month in date and type in types:
            titles_list.append(title)
            brands_list.append(brand)

            brand_count += 1
            count += 1
        
    # print("Total news on this month for " + brand + ": " + str(brand_count))
print("\nTotal news on this month: " + str(count))

    


Total news on this month: 103


In [ ]:
date = "Mei 2022"
df = pd.DataFrame({'Title':titles_list, 'Brand':brands_list})
df.to_csv(f"../../CSV/{date}/viva.csv", index=False)